In [15]:
MAX_M = 64
MAX_N = 64
MAX_TURN = 9999

def make_board(x):
    # 文字列からボードを作成する。
    res = []
    s = x.split('\n')
    for row in s[2:len(s)-1]:
        res.append(list(row))
    return res

def d_copy(b):
    # 二次元配列のディープコピー
    return [row.copy() for row in b]

In [27]:
# 深さ優先探索：失敗

class RoutD(object):
    def __init__(self ,b):
        self.b = b
        self.row_num = len(b)
        self.col_num = len(b[0])
        #self.memo = [[{'u':9999,'b':9999,'r':9999,'l':9999,'s':0} for i in range(self.col_num)] for j in range(self.row_num)]
        self.memo = [[9999 for i in range(self.col_num)] for j in range(self.row_num)]

        #下、右、左、上 
        self.dirs = [(1, 0, 'b', 'u'),(0,1,'r', 'l'), (0, -1, 'l', 'r'),(-1, 0, 'u', 'b')]
        
        self.stack = []
        
        self.min_cost = 9999
    
    def is_inboard(self, pos):
        # ボード内かどうか
        return (pos[0] >= 0 and pos[0] < self.row_num) and (pos[1] >=0 and pos[1] < self.col_num)
    def is_wall(self, b, pos):
        # 通行不可かどうか
        return b[pos[0]][pos[1]] == '#'
    
    def minimum_turn(self):
        self.search()
        return self.min_cost
    
    def search(self):
        self.stack.append({'m':0,'n':0,'c': 0, 'f':'s', 'b': self.b})
        while len(self.stack) > 0:
            curr = self.stack.pop()
            m = curr['m']
            n = curr['n']
            c = curr['c']
            f = curr['f']
            b = curr['b']

            if m == self.row_num -1 and n == self.col_num - 1:
                # goal
                if c < self.min_cost:
                    self.min_cost = c
                continue

            curr_b = d_copy(b)
            curr_b[m][n] = '#'
            for d in self.dirs:
                # 進む方向
                direction = d[2]
                # 逆方向
                back_direction = d[3]
                #次のポジション
                next = (m + d[0], n+d[1])
                if self.is_inboard(next) is False:
                    continue
                if self.is_wall(curr_b, next):
                    continue
                    
                cost = c + 1
                if direction == f or f == 's':
                    cost = c
                #if cost >= self.memo[next[0]][next[1]][direction] or cost >= self.min_cost:
                if cost >= self.memo[next[0]][next[1]] or cost >= self.min_cost:
                    # コストが最小値を超えたらやめる
                    continue
                else:
                    #self.memo[m][n][f] = cost
                    self.memo[m][n] = cost
                    self.stack.append({'m':next[0],'n':next[1],'c': cost, 'f':direction, 'b': curr_b})
        return

In [41]:
#幅優先探索：失敗
import heapq as hq

class RoutW(object):
    def __init__(self, b):
        self.pq =[]
        #board情報設定
        self.b = b
        self.row_num = len(b)
        self.col_num = len(b[0])
        # 上、下、右、左 
        self.dirs = [(-1, 0, 'u', 'b'),(1, 0, 'b', 'u'),(0,1,'r', 'l'), (0, -1, 'l', 'r')]
        self.memo = [[9999 for i in range(self.col_num)] for j in range(self.row_num)]
        
    def minimum_turn(self):
        #スタートをプッシュ
        hq.heappush(self.pq, (0 , (0 , 0), 's'))
        self.search()
        return self.memo[self.row_num-1][self.col_num-1]
    
    def is_inboard(self, pos):
        # ボード内かどうか
        return (pos[0] >= 0 and pos[0] < self.row_num) and (pos[1] >=0 and pos[1] < self.col_num)
    
    def is_wall(self, pos):
        # 通行不可かどうか
        return self.b[pos[0]][pos[1]] == '#'
        
    def search(self):
        while len(self.pq) > 0:
            curr = hq.heappop(self.pq)
            c = curr[0] #コスト
            # 座標
            m= curr[1][0]
            n = curr[1][1]
            f = curr[2]
            if m == self.row_num -1 and n == self.col_num -1:
                # goal
                return
            
            for d in self.dirs:
                # 進む方向
                direction = d[2]
                # 逆方向
                back_direction = d[3]
                #次のポジション
                next = (m + d[0], n+d[1])
                if self.is_inboard(next) is False:
                    continue
                if self.is_wall(next):
                    continue
                if f == back_direction:
                    #　戻る方向
                    continue
                
                cost = c + 1
                if direction == f or f == 's':
                    cost = c
                
                if self.memo[next[0]][next[1]] <= cost:
                    continue
                
                self.memo[next[0]][next[1]] = cost
                hq.heappush(self.pq,(cost, next, cost))

In [106]:
#ダイクストラ
import heapq as hq

class Node(object):
    def __init__(self, m=0, n=0):
        self.m = m
        self.n = n
        self.cost = 9999
        self.done = False
        #self.direction = ''
        self.direction = []
    
    def __eq__(self, other):
        return self.m == other.m and self.n == other.n
    
    def __lt__(self,other):
        if self.cost != other.cost:
            return self.cost < other.cost
        # 進んでいる方が前に来るようにする
        #return (self.m, self.n) >  (other.m, other.n)
        return (self.m, self.n) <  (other.m, other.n)

    def __str__(self):
        return '[{0}][{1}],cost={2}, direction={3},done={4}'.format(self.m, self.n, self.cost, self.direction,self.done)
    
def debug(s):
    pass
    #print(s)
        
class Rout(object):
    def __init__(self, b):
        self.pq =[]
        #board情報設定
        self.b = b
        self.row_num = len(b)
        self.col_num = len(b[0])
        self.memo = [[Node(j, i) for i in range(self.col_num)] for j in range(self.row_num)]
        
        #下、右、左、上 
        self.dirs = [(1, 0, 'b', 'u'),(0,1,'r', 'l'), (0, -1, 'l', 'r'),(-1, 0, 'u', 'b')]
        
    def minimum_turn(self):
        self.search()
        return self.memo[self.row_num-1][self.col_num-1].cost
    
    def is_inboard(self, pos):
        # ボード内かどうか
        return (pos[0] >= 0 and pos[0] < self.row_num) and (pos[1] >=0 and pos[1] < self.col_num)
    
    def is_wall(self, pos):
        # 通行不可かどうか
        return self.b[pos[0]][pos[1]] == '#'
    
    def get_cost(self,curr, d):
        # コストの算出
        #if curr.direction == '' or curr.direction  == d:
        if len(curr.direction) == 0 or d in curr.direction:
            return 0
        return 1
    
    def search(self):
        #スタートをプッシュ
        self.memo[0][0].cost = 0
        hq.heappush(self.pq,self.memo[0][0])
        
        while len(self.pq) > 0:
            curr = hq.heappop(self.pq)
            
            debug("pop")
            debug(curr)
            
            curr.done = True
            #if curr.m == self.row_num -1 and curr.n == self.col_num -1:
            #    # goal
            #    debug('goal')
            #    return
            
            for d in self.dirs:
                #次のポジションの座標
                next_m = curr.m + d[0]
                next_n = curr.n + d[1]
                # 方向
                #direction = d[2]
                dr = d[2]
                # 来た方向
                # front = d[3]
                if self.is_inboard((next_m,next_n)) is False:
                    continue
                if self.is_wall((next_m, next_n)):
                    continue
                
                nx = self.memo[next_m][next_n]
                #cost = curr.cost + self.get_cost(curr, direction)
                cost = curr.cost + self.get_cost(curr, dr)
                
                debug("child")
                debug(nx)
                
                if nx.done:
                    if cost == nx.cost:
                        if dr not in nx.direction:
                            nx.direction.append(dr)
                    elif cost < nx.cost:
                        nx.cost = cost
                        #nx.direction = direction
                        nx.direction.clear()
                        nx.direction.append(dr)
                        nx.done = False
                        hq.heappush(self.pq, nx)
                        debug('push-1')
                elif nx in self.pq:
                    if cost == nx.cost:
                        if dr not in nx.direction:
                            nx.direction.append(dr)
                    elif cost < nx.cost:
                        self.pq.remove(nx)
                        nx.cost = cost
                        #nx.direction = direction
                        nx.direction.clear()
                        nx.direction.append(dr)
                        hq.heappush(self.pq, nx)
                        debug('push-2')
                else:
                        nx.cost = cost
                        #nx.direction = direction
                        nx.direction.clear()
                        nx.direction.append(dr)
                        hq.heappush(self.pq, nx)
                        debug('push-3')
                

In [107]:
b = [
    ['.', '.' ,'#'],
    ['.', '.' ,'.']
]
print(Rout(b).minimum_turn())
#print(RoutD(b).minimum_turn())
#print(RoutW(b).minimum_turn())

b = [
    ['.', '.' ,'#'],
    ['#', '.' ,'.']
]
print(Rout(b).minimum_turn())
#print(RoutD(b).minimum_turn())
#print(RoutW(b).minimum_turn())

b = [
    ['.', '.' ,'.'],
    ['#', '.' ,'#'],
    ['.', '.' ,'.']
]
print(Rout(b).minimum_turn())
#print(RoutD(b).minimum_turn())
#print(RoutW(b).minimum_turn())

1
2
2


In [108]:
s = """
10 15
..##.....#.....
..........##...
.##...##..#....
#....#.......#.
.......#.......
.....#.#..#....
.....#.#..#....
.#............#
...#.##....##..
..........#....
"""
b = make_board(s)
# 期待値5
print(Rout(b).minimum_turn())
#print(RoutD(b).minimum_turn())
#print(RoutW(b).minimum_turn())


5


In [109]:
s="""
30 20
.##.##..#..#.#......
....#..#.#.#...#...#
###....#....#...#.#.
##.........#..#..#..
..##.#.##...#..#....
.###.....##.#.###...
...........#..#.###.
.#.##...#...#...#...
.#.##..#.#.#.....##.
...#...#..#...#..#..
..###...###..#..#..#
.#..#.#...###.#.#...
#...#..#....#..##.#.
.#...#.#..##.##....#
.#...#.#........#.##
............#.#..#..
...#.#...#...#.....#
....##....#..##.###.
.#..#..###.#..##....
.#.##...###...#.....
.#..#..#.#........#.
#....##.###.###.#..#
#....#.#.##.#.#.##..
.......#.##..#......
........##.....#####
..####..#..#.####...
#.#.#...###...###.##
##..###.#.##...##.##
..#.#....#..##......
..##..#..#..##...#..
"""
b = make_board(s)
# 期待値20
print(Rout(b).minimum_turn())
#print(RoutD(b).minimum_turn())
#print(RoutW(b).minimum_turn())

20


In [110]:
s="""
40 40
...##......#...#.##..#.##......##.......
.....##.#...###.#..#......###..#......#.
...##.#........###.........##..#..#.....
#......##...###.....#..#...#.....#.##.#.
..#.......#...#......#.......#....#..#.#
.........#.##.##.#.##..#.##.#..#..#..##.
.........#...#.#.#.....#............#...
.#...#.....#....#.#...#.......#......#..
#.#......#..##..#..#....#..............#
.##..#.....#.#.#.......#.....#.##.....#.
.........#....#....#......#.......#....#
#..#...........#.........#....#.....#...
##.#...#.#.##.....#.#..#.#..##......#...
.#...##.......#..#.....#..#..#...###....
.#.#.....##....##.####...#.###.....##..#
........#.##..#.#.....#...#.#...#....#..
..###.#..#......##..#.....#.##....#....#
....#....#...#...#.............##.#....#
..##..#..#..###..............#..#.#..#..
....#.##.#.....##.##...###..#..#.#...##.
..#......#...#..#..#...#.#.#.####...#...
...........#.......#......#....#.....#..
#....#....#....#.....#....##..###..#....
.........#...#.......#.......#.#.....#..
.....#..#..#..#............#..#....##...
..##......#..#......#.##....##.##..#..#.
....##....###.......#.#.....#..#....#...
###.#....#.......#.##.#............#....
.........#....#...#......##.#.....#...#.
....#.##........#......#..##.#.........#
#.....###...#.#.###.#...........##...#..
##....##.........###..##.#.##....#.#....
#.....##.........#.#.#.##....#.##..#....
..##.#..........#.#...#..#.##...........
#.#..#..#.......#..###..#..#...........#
#...........##..#...#......#.#......#.##
.....#......#.#............#..........##
##......##........#...#......####...#...
......#....##.#..##..#.#............##..
##..#.......##...........##.##......##..
"""
b = make_board(s)
# 期待値17?
print(Rout(b).minimum_turn())
#print(RoutD(b).minimum_turn())
#print(RoutW(b).minimum_turn())

17


In [111]:
s = """
64 64
.......#...........#.#....#..#....###...#.##..............#.....
#....#.....#...#......###........##...#.............##..#.......
#....#..##..#......##.#.#....#.##...#........#.....#.#........#.
..#.##.....#.........##.........#..#..#..###.#.......##......#..
..#....##...##...#...##..........#..#...........##.#.###.#...#.#
.#..##....#..##..........#.#.#....#...#..#.##.....#.##.........#
..#.....#....##....#.#.#...#....#...#...##....###..#....#.#..#..
.#.#....#.#.......##.#.....#...............#.#####.......#...#.#
..#.#.##.#....#.#.#.#.......#...#......#.......#..#....#..#.#...
.##...........##........#...#.......#.....#...#.....#..#.......#
...#.......#.....#.###..............#..#..#...##.#..#.#.#.###...
........#.#..#.##.....#.#..#...#####....#.......#.##.#.##.#.....
.#...##.....#.#.......##......#.....#...#...#....#.......##....#
...................#.#....#.#.....####...##.....................
#....##......##.#...#..#.......##.#...#..#..#...........#.#.....
####.....................#.....#...#.#...#..#........#..........
.......#...#..........#....#...........#...#.##..#...###..#....#
...#.....##.##...#............#........#.#.......#.....##.#...#.
#..##.....#..#.#...#.#.#...#...#.##.#...#...#....#.........##...
.....#.#.#.......#....#.....#....#..#....####......#...#........
..#..#...........#...#....#...#...#.#..............#....#....##.
.#.#.#.........#...........##....#..............##.....###...#..
............#....#..##........##..#....#...##..#..#....#....#..#
###.....#..#....#.#.##.....#...#..##....##............#.#..#.#.#
......#...#....#....#.....#....##..#.###....###..#....#........#
..#.##......##.#.##.#...#......######........##.#....##.#.#.#..#
...#...#........#.....####.#.##.#....#..##...#.......#..........
........#..#..#.#.....#......#...#..#..#......#....##.#.#..###..
#....##.......#...#...#......#.#.....#..#..##...#...#...........
#..#.......#.......#.##.#...#..#.......#.#......#.##............
...#...........###......#.......#...........#..........#.......#
......#...........#.....#..##........#...#...#....##.#.#....#...
....##...#...#.#.....###.#.#.#.....#..#.#.#...#....#.#........#.
.#.#..#..##...........#.....##.##.#....##....##...#....#..#.##..
#.........#.......##...........#......##...##.#.##.##.....##....
...##....##.##.#.###.#....#.....##......#..#..........#.##.....#
........##..####.......#......##.....#...##.....#...........#...
.#.#....##..#..#..##.....#.......##............#....#...#..#.##.
.........#.....#.....###....#........#...#....##..#.....#...#...
...#...#...##......#...###....#.....#...#.#..##..#......#..#....
..........#.#...#.##......#...##.##.............#..##.....##.#..
.................##...#.....#.#.##.....#.......#.....#..#.....#.
.....#..................#...#....#.#.#...#......#...##..#.#....#
..#....#.#.#.#...#.....#......##...............####....##.......
#..#.#..#.......#.........#......#................#...#.###..##.
.##..........#.......#..#....##........####.....#...#.#.#.#....#
...#.##.#..##.......#.....##......#.#.....#.#.........#.#.#.....
..#.......##.......#...................#..#..#.....#.........##.
##...###....#...#..#...#........#..#.#.......##.....#....#..#...
...##..#.........##...#.........#..#.#.#......#.......#..#..##.#
.#...#....#.#.....#....#............#.#...#...#..#......#.##....
........#...##..#.#.#.....#.#.#......#.##.............#.###.....
#....#.#..#....#..#.##........#........#.....#.#.#....#..#...###
..#......#.#.........#.#....#.#.....#....##.#..#.##.....#.#.....
.##......#...#......#.#...#.....##...........#........#.........
...#...##.#...#.#....#.......###....#.......#...#.#....#.####.#.
.#...#.......#...#.....##.#..##.................#.....##.#..#...
...#..#.................#....#.##........#....#.....##..#..##.#.
#..#.......#.#.#.#.#..###........#.....#.#..#...#...##.#....#...
###.#.#.#.#..#.........#.....#.........#.#..##.#.........#...##.
#.....##.#....#.##.##.#..##.##...##...##.##...#.#..#.##.#.....#.
.##..#..#...#..##.#.##......#.#..........#...........#.......#..
..#.......#...................##.......#.#..#..#......#...#.....
......#.#..#.......##...#.##.....#..##..#..#........#.#.....#.#.
"""
b = make_board(s)
# 期待値21
print(Rout(b).minimum_turn())
#print(RoutD(b).minimum_turn())

21


In [112]:
s="""
64 64
.#..#...#..###.#.##.....##...##.#..#.###.#.#...#.#.##....#.#.#..
..#.#.#....#.##..#...#.........####.#..#....####.....##.#..##.##
.....##..#......##..#..#..#..##...#.....##....###..###......#.#.
.....##.....###.....#.#.#.#..###..#.......#.......##.#..##..##..
.#.#...........#..##.##..#.#..###.##........#..#.#..#.##.....#..
#..##...#...#.........#.#...##......####.#...#.##.......###...##
......#..............#...#.###.....#.#...#.#......###.##....#.##
##.......#.##..##..#...#..##..#..##...#.#.#..#........#.###...##
......#...#.#.#...#......###.#....#..#.#...###....##......#.#.#.
.##.##...#.##......##..###..#...#........#.#..###.#....##..##..#
#..#.#...###..#..#..#.......#.#.##.#.#.#.......#.#.....#.#..#...
.##..#.##...##...#..#..#.#..#....#..#.##.....#.......###.#.##..#
#.....#.#..........#..#.###..##.#.#...#.#...#..#..#..#.##...#...
.##..#..#.#....##...##..###....##.#..##.#..##...#..####...#.....
.#....#..#.##....#.#.##..###...##.#.#.#.......#....##......#..#.
.........#..#.#........#...#......#.........#.#.#..#.#....####.#
#####.##....###.....#.##...#..##....#.........###..#...##..###..
..##....#..#.##..#......#.....#.##.#.###.#.##......#.....#.#....
...#####...#..##.#..#..####...###..###.#......####.#....#.##..#.
..#.......#.##..#..###.......###...####.##..###.#...##.#.##...#.
..##....#.....#.##...###...#..#..#..#.###...#.....#.#..#.#.....#
####......##..##...#...##.##..........#####.#......#...##....##.
##.......##....#..#....#......#.....##..#.##..##.#....#.##.#####
..##.......#.###.....#..#.......##...#....#...#...#####.......#.
##.....#...#........##....#....#..##.....#.#...#...#............
..##............##.###...#.#......#.......##.##..#....#........#
##.#..#.....##...#...#.#.#..#......#.....######...#......#...##.
......##....#..#......#.......##..#.#.....#.#..#.##.#.##.....#..
.#.#....#.#...#..####..#..#..##....###........#..#...#...##.#..#
#.##....#.....#...#.#.......#....#..#..........##....#.###......
#.#.....#.#...#.......#.###..##...##.#..........................
..#.###.......##...##..#.#..#.#.....#....#......#...###.##......
..#....##...#.##......#.........#...#...#...###..#.####.#...###.
#.....##..#.#.#..#..###..#...##..#.#........#####.#....#.#..#...
..#.#.#.....#....#..#.###.....#.....##...##........#.##...###...
.......#....#.##.#..#....#.#.#.#.##.#..#...#...#...#.......##...
.#.##.#.##...###............#.#..#.....#..###.##.#.........#....
..#.....#.###......##.#...#.....#.##....#.......#......#.###.#.#
....######.#.#...#.#..#..#.#....#..#.#.###...........#.##.....#.
.####.##...#...##..#....#..#...#...#..##.##..#.....##...#.##...#
..###.#.#...#.....#...#...##....##.##....##.##..#.#.###.......#.
#...#.......##.##.##.##..#.#..#####....#.##.#.....#...##...##...
.#.#...##..#...#.##.....#.#......#.#..#.#....##..#........#.##..
..###..#.###...#.#..#......#..####.#.#..####.#######...##..#.#..
...#...#.###..##.#...#..#...........##..#...#..####..#..#.###.##
.....#.#.###.#.#.#.....#...#.##..#.#.##...###.#.#....#...#......
##..##..##....#...##....#..##.#....#....#.#.......#....#..#...#.
...#.....#.#.#.#..#.#..#.#...#...#.#....#...###...#....#......##
..#...#..###..#####....##........#..#..#........#...#..#........
##.##.####.###....#....#.....######.#..#.#.#.#...#..#..##.#...#.
...#.#.##.#.#.##.#..#.......#..#...#.#.#.#...#.......##.....#.#.
....#.##....##...#..#...##.#..#..#......#..##..##.#.#....#.##...
...#.###...#...#.##...##..#.#.###...#..........#......#..##..#..
#..#..#..#..#.##......##...#..###.#..#.....#......#.......#....#
..#.......####....#.#.#........#.....#.....#....#..##...#......#
.........#...#.#.....#####.#..#..........#.......##..##...####..
#.#.###....#....#.#..##.#.....#..#..#.#..##.#.#......##......###
#.#..#.#...#......#.#....#..#.#.#..#.##......##..#..##.....#..#.
.#.....#..#..##.......#.#.##.....#.##.#........##..........#.##.
#.#......#..#..#.#...##......#.#.....#.....#..####.....#..#.....
#.#.#.##......#...#.#.#....##.##..#.#.#.#.#...#.#....#####.....#
...#####...###..#........#..#..#.##..#.#.#.##.#.#..#.#....#.....
...###....#.#..#...#....#.#...##..##..#...#..#..##.###...#.#..#.
..##......#.####......##......#....#..#..#.........#..#..#...#..
"""
b = make_board(s)
# 期待値32
print(Rout(b).minimum_turn())
#print(RoutD(b).minimum_turn())

32


In [2]:
data = input().split()
m = int(data[0])
n = int(data[1])
b = []
for i in range(m):
    b.append(list(input()))

print(RoutD(b).minimum_turn())

3 2
..
.#
##
['.', '.']
['.', '#']
['#', '#']
1
